## **Cat classification**

This notebook is modified from AI Builder's lesson 4v (image_classification.ipynb)

### Preparing data

```
Downloaded format:
data/
    category1
    category2
    category3
```


In [ ]:
import os
import os.path as op
import shutil
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
# get all image paths
img_df = pd.DataFrame(glob("extras/*/*.jpg"), columns=["path"])
img_df["id"] = img_df.path.map(lambda x: op.basename(x).replace(".jpg", ""))

## Image classification with FastAI

In [ ]:
# สำหรับคนที่ยังไม่ได้ลงไลบรารี่ fastbook ให้ uncomment บรรทัด !pip install
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *

In [ ]:
fields = DataBlock(
    blocks=(ImageBlock, CategoryBlock), #
    get_items=get_image_files, #Get list of files
    get_y=parent_label, #Getting the folder name (to see which type of dog it is)
    splitter=RandomSplitter(valid_pct=0.2, seed=314), #Split train/validation 80:20. 
    #If you have already categorized by folder, change the line to `GrandparentSplitter(valid_name='validation')`
    item_tfms=RandomResizedCrop(224, min_scale=0.5), #Resize the image into 224x224 (to make it same resnet34)
    batch_tfms=aug_transforms()
)

In [ ]:
get_image_files("data/")

In [ ]:
dls = fields.dataloaders("data")

In [ ]:
dls.vocab

In [ ]:
dls.train.show_batch(max_n=8, nrows=2)

## Training

In [ ]:
learner50 = cnn_learner(dls, resnet50, metrics=[error_rate, accuracy])

In [ ]:
lr2 = learner50.lr_find() # หา learning rate ที่เหมาะสม

In [ ]:
lr2.valley

In [ ]:
learner50.fine_tune(epochs=10, freeze_epochs=1, base_lr=lr2.valley)

## Extra Information

In [ ]:
interp = ClassificationInterpretation.from_learner(learner50)
interp.plot_confusion_matrix(figsize=(12,12), dpi=150)

In [ ]:
interp.most_confused(min_val=5)

In [ ]:
learner50.show_results(max_n=16)

## Saving the model

In [ ]:
learner50.export("dbc_resnet50_fastai_bigv3.5-cleaned.pkl")

## Loading model for prediction

In [ ]:
from fastbook import *
from glob import glob
from pathlib import Path
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score

learn_inf = load_learner('dbc_resnet50_fastai_bigv3.5-cleaned.pkl')

In [ ]:
!wget https://cdn.discordapp.com/attachments/1009623322851561513/1101061230879309834/cat.png

In [ ]:
Image.open("cat.png").resize((224, 224))

In [ ]:
learn_inf.predict("cat.png")

In [ ]:
!wget https://media.discordapp.net/attachments/1009623322851561513/1101062512310497390/cat2.png

In [ ]:
Image.open("cat2.png").resize((224, 224))

In [ ]:
prediction = learn_inf.predict("cat2.png")